
# Week 13-2 기계학습 오픈 프레임워크
#### Machine Learning with Python by idebtor@gmail.com

-----------------------------------

In [ ]:
# Package imports
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets
import sklearn.linear_model
# Our own private imports
import imp
import joy
imp.reload(joy)

%matplotlib inline
np.random.seed(1)   # a good practice for reproducibility and debugging

# The following code is used for hiding the warnings and 
# make this notebook clearer.
#import warnings
#warnings.filterwarnings('ignore')

## CNN 구현: 신경망 구축 - TensorFlow

### 1. Tensorflow에서 MNIST 자료 읽어오기
TensorFlow에는 mnist 데이터셋을 기본적으로 제공해주고 있으며, 아래와 같이 두 줄이면 데이터를 불러올 수 있습니다. 

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data/", one_hot=True)

### 2. Tensorflow에서 연산에 사용할 변수와 함수 정의
Tensorflow의 모델에서 연산을 실행할 때 값을 입력할 공간, placeholder와, 필요한 함수들을 정의합니다.

In [ ]:
import tensorflow as tf

x = tf.placeholder(tf.float32, [None, 784])

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

### 3. Tensorflow에서 신경망 구축
MNIST 데이터를 학습하기 위한 신경망을 구축합니다. 
입력층과 2개의 은닉층, 1개의 출력층으로 구축합니다.

In [ ]:
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
dropout_ratio = tf.placeholder(tf.float32)

x_train = tf.reshape(x, [-1, 28, 28, 1])

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

h_conv1 = tf.nn.relu(conv2d(x_train, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, dropout_ratio)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_hat = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

### 4-1. Tensorflow에서 신경망 학습
신경망을 학습 시키기 위해서, 모델의 성능을 확인할 비용과 손실을 담는 변수를 정의합니다. 또한 모델의 손실을 정의하기 위해 Cross Entropy를 이용합니다. 

In [ ]:
CE = tf.reduce_mean(
        -tf.reduce_sum(y * tf.log(y_hat), reduction_indices=[1])
     )
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(CE)
correct_prediction = tf.equal(tf.argmax(y_hat,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for i in range(10):
        batch = mnist.train.next_batch(32)
        if i%100 == 0:
            train_accuracy = accuracy.eval(session=sess, feed_dict={
                x:batch[0], y: batch[1], dropout_ratio: 1.0
            })
            print("step %d, training accuracy %g"%(i, train_accuracy))
        train_step.run(feed_dict={x: batch[0], y: batch[1], dropout_ratio: 0.2})
    
    acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels, dropout_ratio: 1.0})
    print("Test accuracy: {}".format(acc))

### 4-2. Tensorflow에서 신경망 학습

In [ ]:
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

y_hat = tf.nn.softmax(tf.matmul(x, W) + b)
y = tf.placeholder(tf.float32, [None, 10])
CE = tf.reduce_mean(-tf.reduce_sum(y * tf.log(y_hat), reduction_indices=[1]))

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(CE)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for i in range(1000):
    batch_x, batch_y = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_x, y: batch_y})

for i in range(1000):
    batch_x, batch_y = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_x, y: batch_y})

### 5. Tensorflow에서 모델 평가
tf.argmax(y_hat, 1)을 통해 판단한 예측 값과, tf.argmax(y, 1)을 통해 실제 레이블을 값을 비교하여 정확히 학습했는지 판단합니다.

In [ ]:
correct_prediction = tf.equal(tf.argmax(y_hat,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print("Test accuracy: {}".format(sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels})))

## CNN 구현: 신경망 구축 - Keras

### 1. Keras에서 MNIST 자료 읽어오기
Keras에는 mnist 데이터셋을 기본적으로 제공해주고 있으며, mnist.load_data()를 통해 데이터를 얻어올 수 있습니다. 자료를 읽어오고 전처리과정도 수행해줍니다.

In [ ]:
from keras.datasets import mnist
from keras import backend as K
from keras.utils import np_utils
import numpy as np

img_rows, img_cols = 28, 28

(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

### 2. Keras에서 신경망 구축
입력층과 2개의 은닉층 1개의 출력층으로 구성된 신경망을 구축합니다.

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

# define the model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

# summarize the model
model.summary()


### 3. Keras에서 신경망 모델 컴파일
구축한 신경망이 Cross Entropy를 가지며, optimizer로는 Adadelta를 사용하도록 합니다. 또한 이 신경망은 정확도를 측정하도록 컴파일 합니다.

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

### 4. Keras에서 신경망 모델 학습
신경망 모델이 학습하도록 합니다. checkpointer를 사용함으로서, 매 epochs마다 모델을 저장하도록 합니다.

In [ ]:
from keras.callbacks import ModelCheckpoint   

# train the model
checkpointer = ModelCheckpoint(filepath='mnist.model.best.hdf5', 
                               verbose=1, save_best_only=True)
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test),
         callbacks=[checkpointer])

### 5. Keras에서 신경망 분류 정확도 측정
신경망이 제대로 학습했는지 평가합니다

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

## CNN 구현: 신경망 구현 - PyTorch

### 1. Pytorch에서 MNIST 자료 읽어오기
Pytorch에는 mnist 데이터셋을 기본적으로 제공해주고 있으며, torch.utils.data.DataLoader()를 통해 데이터를 얻어올 수 있습니다. 자료를 읽어오고 전처리과정도 수행해줍니다.

In [ ]:
import torch
from torchvision import datasets, transforms

use_cuda = not False and torch.cuda.is_available()
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
        batch_size=64, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, 
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
        batch_size=64, shuffle=True, **kwargs)

### 2. Pytorch에서 신경망 구축 및 순전파 구현
한 개의 입력층과 2개의 은닉층 1개의 출력층을 구성하도록 신경망을 구성하며, 순전파 부분또한 하나의 클래스 안에 메소드로 구현해줍니다.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=2)
        self.conv1_drop = nn.Dropout2d(0.2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=2)
        self.conv2_drop = nn.Dropout2d(0.2)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=2)
        self.conv3_drop = nn.Dropout2d(0.2)
        
        self.fc1 = nn.Linear(256, 10)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1_drop(self.conv1(x)), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = F.relu(F.max_pool2d(self.conv3_drop(self.conv3(x)), 2))
        x = x.view(-1, 256)
        x = self.fc1(x)
        
        return F.log_softmax(x)

### 3. Pytorch에서 신경망에서 사용하는 변수 정의
신경망에서 사용하는, 반복 횟수(epochs), 학습률(learning_rate) 등의 변수들을 정의합니다.

In [ ]:
n_epochs = 10
learning_rate = 0.01
random_seed = 1
log_interval = 10

network = Net()
optimizer = optim.RMSprop(network.parameters(), lr=learning_rate)

### 4. Pytorch에서 신경망의 학습 메소드 정의
신경망에서 데이터를 학습하도록 함수를 정의합니다. optimizer로는 zero_gradient를 사용하며 Cross Entropy를 이용하여 손실 함수를 최소화 합니다.

In [ ]:
def train(epoch):
    network.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = network(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            

### 5. Pytorch에서 신경망의 테스트 메소드 정의
신경망에서 데이터를 테스트 하도록 함수를 정의합니다. 얼마나 정확하게 학습을 수행했는지 알아보도록 합니다.

In [ ]:
def test():
    network.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        output = network(data)
        test_loss += F.cross_entropy(output, target, size_average=False).item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).sum()
        test_loss /= len(test_loader.dataset)
    print('Test Accuracy {}%\n'.format(100.*correct / len(test_loader.dataset)))

### 6. Pytorch에서 신경망의 분류 정확도 측정
실제 모델 학습 평가를 확인하는 코드입니다.

In [ ]:
for epoch in range(1, n_epochs+1):
    train(epoch)
    test()

----------
Rejoice in the Lord always. I will say it again: Rejoice! (Ph4:4)